In [27]:
import numpy as np
import trimesh
import tqdm 

In [28]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    # -------- TODO -----------

    vector_a = vertices[faces[:, 0]] - vertices[faces[:, 1]]
    vector_b = vertices[faces[:, 0]] - vertices[faces[:, 2]]
    area = np.linalg.norm(np.cross(vector_a, vector_b), axis=1) / 2.0

    prob = area / np.sum(area)
    sampled_faces = np.random.choice(faces.shape[0], size=sample_num, p=prob)
    sampled_faces = faces[sampled_faces]

    # use the alternative method in slides to sample points
    r1, r2 = np.random.rand(sample_num), np.random.rand(sample_num)

    uniform_pc = (
        (1 - np.sqrt(r1))[:, None] * vertices[sampled_faces[:, 0]]
        + (np.sqrt(r1) * (1 - r2))[:, None] * vertices[sampled_faces[:, 1]]
        + (np.sqrt(r1) * r2)[:, None] * vertices[sampled_faces[:, 2]]
    )

    return area, prob, uniform_pc
        

In [29]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    # -------- TODO -----------

	# init the distance with infinity
	distance = np.full((pc.shape[0],), np.inf)

	results = []
	p = pc[np.random.randint(pc.shape[0])]
	results.append(p)

	for i in tqdm.tqdm(range(sample_num - 1)):
		# update the distance
		distance = np.minimum(distance, np.linalg.norm(pc - p, axis=1))
		index = np.argmax(distance)
		results.append(pc[index])
		p = pc[index]

	results = np.array(results)
	return results

In [30]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [31]:
def save_point_cloud_to_obj(pc, filename):
    """
    Save point cloud to an .obj file.
    Args:
        pc: numpy array of shape (N, 3), where N is the number of points.
        filename: str, the output .obj file path.
    """
    with open(filename, "w") as f:
        for point in pc:
            f.write(f"v {point[0]} {point[1]} {point[2]}\n")
    print(f"Point cloud saved to {filename}")

save_point_cloud_to_obj(uniform_pc, "../results/uniform_pc.obj")


Point cloud saved to ../results/uniform_pc.obj


In [32]:
# task 2: FPS
init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

100%|██████████| 511/511 [00:00<00:00, 19556.81it/s]


In [33]:
save_point_cloud_to_obj(fps_pc, "../results/fps_pc.obj")

Point cloud saved to ../results/fps_pc.obj


The results of the point cloud sampling methods, which is visualized using MeshLab, are shown below. 
<div style="display: flex; justify-content: space-around;">
    <div>
        <img src="../results/uniform_pc_visual.png" alt="Uniform Sampling" style="width: 100%;">
        <p style="text-align: center;">Uniform Sampling</p>
    </div>
    <div>
        <img src="../results/fps_pc_visual.png" alt="FPS Sampling" style="width: 100%;">
        <p style="text-align: center;">FPS Sampling</p>
    </div>
</div>

In [38]:
# task 3: metrics

from earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------

CD, EMD = np.zeros(5), np.zeros(5)
for i in range(5):
    # Uniform sampling
	_,_, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)
	# FPS
	_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
	fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

	distance = uniform_pc[:, None] - fps_pc[None, :]

	cd_distance = np.mean(np.min(np.linalg.norm(distance, axis=2), axis=1)) + np.mean(np.min(np.linalg.norm(distance, axis=2), axis=0))
	cd_distance /= 2.0		# for fair comparison, we divide by 2

	# convert numpy array to list of tuples
	uniform_pc_list = [tuple(point) for point in uniform_pc]
	fps_pc_list = [tuple(point) for point in fps_pc]
	emd_distance = earthmover_distance(uniform_pc_list, fps_pc_list)
	CD[i], EMD[i] = cd_distance, emd_distance


CD_mean = np.mean(CD)
CD_var = np.var(CD)
EMD_mean = np.mean(EMD)
EMD_var = np.var(EMD)

print(f"CD mean: {CD_mean}, CD var: {CD_var}")
print(f"EMD mean: {EMD_mean}, EMD var: {EMD_var}")

# # For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

100%|██████████| 511/511 [00:00<00:00, 8808.05it/s]


move 0.001953125 dirt from (np.float64(25.661853158194592), np.float64(7.48153794088962), np.float64(26.265777237013317)) to (np.float64(25.83188047614601), np.float64(7.5588686897225745), np.float64(26.66570576780395)) for a cost of 0.0008621053999459943
move 0.001953125 dirt from (np.float64(27.33166968579684), np.float64(40.0515023285721), np.float64(29.594824361372176)) to (np.float64(27.289884308656056), np.float64(42.44826873116881), np.float64(31.808490065769057)) for a cost of 0.006372861932143954
move 0.001953125 dirt from (np.float64(53.79072639616169), np.float64(28.422066100376977), np.float64(31.027841798915368)) to (np.float64(52.85890119339722), np.float64(29.203122997850006), np.float64(33.177122972889244)) for a cost of 0.004822976200999606
move 0.001953125 dirt from (np.float64(45.872063427623395), np.float64(34.55263722969764), np.float64(30.540536744507726)) to (np.float64(44.36245872574669), np.float64(34.75674955274331), np.float64(30.641743136079636)) for a cost 

100%|██████████| 511/511 [00:00<00:00, 33343.54it/s]


move 0.001953125 dirt from (np.float64(38.43137263630211), np.float64(20.751099914137214), np.float64(20.760715003826625)) to (np.float64(40.09635778281827), np.float64(19.143678788932025), np.float64(21.104763496682825)) for a cost of 0.004569789791035559
move 0.001953125 dirt from (np.float64(38.60032690338174), np.float64(23.919212999887364), np.float64(42.32689942245226)) to (np.float64(37.70689976896762), np.float64(23.250471595093728), np.float64(42.410407525490825)) for a cost of 0.0021857560660277094
move 0.001953125 dirt from (np.float64(40.990582905837115), np.float64(30.14665512177601), np.float64(22.98521899258324)) to (np.float64(49.9844037208246), np.float64(30.49396324983207), np.float64(25.778853133657904)) for a cost of 0.018406462585785686
move 0.001953125 dirt from (np.float64(23.97505006480799), np.float64(43.37305351489479), np.float64(25.331747389040487)) to (np.float64(22.768663839329818), np.float64(44.17813503602767), np.float64(24.73206794977654)) for a cost o

100%|██████████| 511/511 [00:00<00:00, 32292.07it/s]


move 0.001953125 dirt from (np.float64(52.44927132030046), np.float64(19.270460841765953), np.float64(28.77055247089896)) to (np.float64(52.43966407193865), np.float64(19.22080112952367), np.float64(28.86901078376601)) for a cost of 0.00021619272533054287
move 0.001953125 dirt from (np.float64(49.46634174167247), np.float64(32.6927960451785), np.float64(28.77653569968315)) to (np.float64(51.42273999197716), np.float64(30.476675701491175), np.float64(34.733673832425865)) for a cost of 0.012988821391505541
move 0.001953125 dirt from (np.float64(30.9493065842056), np.float64(37.20862338203858), np.float64(35.105198525067244)) to (np.float64(32.47125591302826), np.float64(36.7287261469516), np.float64(36.116215595814026)) for a cost of 0.0036896938370066263
move 0.001953125 dirt from (np.float64(9.029371106604874), np.float64(41.24932297661366), np.float64(26.20148305170417)) to (np.float64(14.209420161603068), np.float64(38.497368120634505), np.float64(22.043631528148904)) for a cost of 0

100%|██████████| 511/511 [00:00<00:00, 36957.09it/s]


move 0.001953125 dirt from (np.float64(27.123231862750416), np.float64(7.305370022264918), np.float64(39.37343928708248)) to (np.float64(25.748041578903255), np.float64(7.210521369006995), np.float64(36.878285789286736)) for a cost of 0.005567583332705629
move 0.001953125 dirt from (np.float64(44.567236122998), np.float64(33.20181403508903), np.float64(26.17159712118573)) to (np.float64(42.686006670671084), np.float64(33.79156312705453), np.float64(27.029654862415608)) for a cost of 0.004199487332917263
move 0.001953125 dirt from (np.float64(23.901176098508564), np.float64(10.395245595838027), np.float64(27.695392946090912)) to (np.float64(23.568015218446085), np.float64(12.015305910459071), np.float64(27.81288538244676)) for a cost of 0.003238535727956478
move 0.001953125 dirt from (np.float64(26.331422892341244), np.float64(21.281429180943977), np.float64(42.50986324072687)) to (np.float64(27.305317188898343), np.float64(22.722545256737284), np.float64(42.48671122160221)) for a cost 

100%|██████████| 511/511 [00:00<00:00, 36153.52it/s]


move 0.001953125 dirt from (np.float64(11.342212938794791), np.float64(40.75982095594857), np.float64(39.75876281895353)) to (np.float64(12.286541521588994), np.float64(39.508789844545504), np.float64(40.56840053253097)) for a cost of 0.003445673732362389
move 0.001953125 dirt from (np.float64(14.450808906820892), np.float64(31.405275955359834), np.float64(37.70631097657716)) to (np.float64(14.474255326236193), np.float64(30.4596239420708), np.float64(33.709226182011506)) for a cost of 0.00802244621350809
move 0.001953125 dirt from (np.float64(8.246689495423794), np.float64(39.805051916543), np.float64(25.433577702820884)) to (np.float64(8.442480201565104), np.float64(40.03715469224598), np.float64(25.373014602511315)) for a cost of 0.0006047550866477912
move 0.001953125 dirt from (np.float64(20.62746941924096), np.float64(50.009822458724805), np.float64(34.819493425773324)) to (np.float64(19.479665502974278), np.float64(51.32196525787195), np.float64(32.12404317567584)) for a cost of 